In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
class ImageRegressor(nn.Module):
    def __init__(self, input_shape=(224, 224, 3), dropout_rate=0.5):
        super(ImageRegressor, self).__init__()

        # Load pre-trained VGG16 model without the top layers
        self.base_model = models.vgg16(pretrained=True).features

        # Freeze the VGG16 layers
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Define the image regressor
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 7 * 7, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_regressor = ImageRegressor().to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(image_regressor.parameters(), lr=0.001)

In [ ]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float).to(device)
x_val_tensor = torch.tensor(x_val, dtype=torch.float).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
# Train the regressor
num_epochs = 20

for epoch in range(num_epochs):
    image_regressor.train()
    train_loss = 0.0
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = image_regressor(inputs)
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    image_regressor.eval()
    val_loss = 0.0

    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = image_regressor(inputs)
            loss = criterion(outputs.view(-1), targets)
            val_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}')


In [ ]:
new_image_tensor = torch.tensor(new_image, dtype=torch.float).unsqueeze(0).to(device)
image_regressor.eval()
with torch.no_grad():
    predicted_modulus = image_regressor(new_image_tensor).item()